
<p style="text-align: center;font-size: 18px;">
    BIO/MIC 494/598 <span style="font-weight: bold;">Microbiome Data Science</span>
</p>
<p style="text-align: center;"><span style="font-size: 32px; font-weight: bold; background-color: #ffc627;">
    Sequencing Data Processing
</span></p>
<p style="text-align: center; font-size: 18px;">
    Team 5
</p>

## Data files

In [1]:
! qiime tools import \
  --type 'SampleData[PairedEndSequencesWithQuality]' \
  --input-path dwdss \
  --input-format CasavaOneEightSingleLanePerSampleDirFmt \
  --output-path demux-paired-end.qza

^C

Aborted!


In [31]:
! qiime dada2 denoise-paired \
  --i-demultiplexed-seqs demux.qza \
  --p-trim-left-f 13 \
  --p-trim-left-r 13 \
  --p-trunc-len-f 150 \
  --p-trunc-len-r 150 \
  --o-table table.qza \
  --o-representative-sequences rep-seqs.qza \
  --o-denoising-stats denoising-stats.qza

## From Qiita to Qiime2

!qiime feature-table tabulate-seqs   --i-data rep-seqs.qza   --o-visualization rep-seqs.qzv


!qiime feature-table summarize   --i-table table.qza   --o-visualization table.qzv   --m-sample-metadata-file metadata.tsv


In [31]:
!qiime phylogeny align-to-tree-mafft-fasttree \
  --i-sequences rep-seqs.qza \
  --o-alignment aligned-rep-seqs.qza \
  --o-masked-alignment masked-aligned-rep-seqs.qza \
  --o-tree unrooted-tree.qza \
  --o-rooted-tree rooted-tree.qza

Saved FeatureData[AlignedSequence] to: aligned-rep-seqs.qza
Saved FeatureData[AlignedSequence] to: masked-aligned-rep-seqs.qza
Saved Phylogeny[Unrooted] to: unrooted-tree.qza
Saved Phylogeny[Rooted] to: rooted-tree.qza


## Taxonomic analysis

In [2]:
!curl -sL \
  "https://data.qiime2.org/2021.2/common/gg-13-8-99-515-806-nb-classifier.qza" > \
  "gg-13-8-99-515-806-nb-classifier.qza"

In [3]:
!qiime feature-classifier classify-sklearn \
  --i-classifier gg-13-8-99-515-806-nb-classifier.qza \
  --i-reads rep-seqs.qza \
  --o-classification taxonomy.qza

!qiime metadata tabulate \
  --m-input-file taxonomy.qza \
  --o-visualization taxonomy.qzv

Saved FeatureData[Taxonomy] to: taxonomy.qza
Saved Visualization to: taxonomy.qzv


# Rarefaction

In [4]:
!qiime feature-table rarefy \
  --i-table table.qza \
  --p-sampling-depth 10102 \
  --o-rarefied-table rarefied_table.qza

Saved FeatureTable[Frequency] to: rarefied_table.qza


Retained 5,849,682 (26.52%) features in 291 (98.64%) samples at the specifed sampling depth (20102).

In [5]:
!qiime feature-table summarize \
  --i-table rarefied_table.qza \
  --o-visualization rarefied_table.qzv \
  --m-sample-metadata-file metadata.tsv

Saved Visualization to: rarefied_table.qzv


In [ ]:
# Taxonomy Barplot

In [83]:
!qiime taxa barplot \
  --i-table table.qza \
  --i-taxonomy taxonomy.qza \
  --m-metadata-file metadata.tsv \
  --o-visualization barplot.qzv

Saved Visualization to: barplot.qzv


### Alpha and beta diversity analysis

In [6]:
!qiime diversity core-metrics-phylogenetic \
  --i-phylogeny rooted-tree.qza \
  --i-table table.qza \
  --p-sampling-depth 10102 \
  --m-metadata-file metadata.tsv \
  --output-dir all_in_one

Saved FeatureTable[Frequency] to: all_in_one/rarefied_table.qza
Saved SampleData[AlphaDiversity] to: all_in_one/faith_pd_vector.qza
Saved SampleData[AlphaDiversity] to: all_in_one/observed_features_vector.qza
Saved SampleData[AlphaDiversity] to: all_in_one/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: all_in_one/evenness_vector.qza
Saved DistanceMatrix to: all_in_one/unweighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: all_in_one/weighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: all_in_one/jaccard_distance_matrix.qza
Saved DistanceMatrix to: all_in_one/bray_curtis_distance_matrix.qza
Saved PCoAResults to: all_in_one/unweighted_unifrac_pcoa_results.qza
Saved PCoAResults to: all_in_one/weighted_unifrac_pcoa_results.qza
Saved PCoAResults to: all_in_one/jaccard_pcoa_results.qza
Saved PCoAResults to: all_in_one/bray_curtis_pcoa_results.qza
Saved Visualization to: all_in_one/unweighted_unifrac_emperor.qzv
Saved Visualization to: all_in_one/weighted_unifrac_em

## Faith Phylogenetic Diversity

In [29]:
! qiime alignment mafft \
  --i-sequences rep-seqs.qza \
  --o-alignment aligned-rep-seqs.qza

Saved FeatureData[AlignedSequence] to: aligned-rep-seqs.qza


In [30]:
!qiime alignment mask \
  --i-alignment aligned-rep-seqs.qza \
  --o-masked-alignment masked-aligned-rep-seqs.qza

Saved FeatureData[AlignedSequence] to: masked-aligned-rep-seqs.qza


In [32]:
! qiime phylogeny midpoint-root \
  --i-tree unrooted-tree.qza \
  --o-rooted-tree rooted-tree.qza

Saved Phylogeny[Rooted] to: rooted-tree.qza


In [33]:
!qiime diversity core-metrics-phylogenetic \
  --i-phylogeny rooted-tree.qza \
  --i-table table.qza \
  --p-sampling-depth 10200 \
  --m-metadata-file metadata.tsv \
  --output-dir core-metrics-results


Saved FeatureTable[Frequency] to: core-metrics-results/rarefied_table.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/faith_pd_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/observed_features_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/evenness_vector.qza
Saved DistanceMatrix to: core-metrics-results/unweighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results/weighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results/jaccard_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results/bray_curtis_distance_matrix.qza
Saved PCoAResults to: core-metrics-results/unweighted_unifrac_pcoa_results.qza
Saved PCoAResults to: core-metrics-results/weighted_unifrac_pcoa_results.qza
Saved PCoAResults to: core-metrics-results/jaccard_pcoa_results.qza
Saved PCoAResults to: core-metrics-results/bray_curtis_pcoa_re

In [1]:
!qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-results/faith_pd_vector.qza \
  --m-metadata-file metadata.tsv \
  --o-visualization core-metrics-results/faith-pd-group-significance.qzv

Saved Visualization to: core-metrics-results/faith-pd-group-significance.qzv


phylogeny-informed alpha diversity using the Faith's PD metric

In [14]:
!qiime diversity alpha-phylogenetic \
  --i-table rarefied_table.qza \
  --i-phylogeny rooted-tree.qza \
  --p-metric faith_pd \
  --o-alpha-diversity faith_pd_vector.qza

Saved SampleData[AlphaDiversity] to: faith_pd_vector.qza


#### Evenness metrics

In [15]:
!qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-results/evenness_vector.qza \
  --m-metadata-file metadata.tsv \
  --o-visualization core-metrics-results/evenness-group-significance.qzv

Saved Visualization to: core-metrics-results/evenness-group-significance.qzv


### Beta Diversity using unweighted UniFrac distances for expermint title (Utilities from A to E )

In [17]:
!qiime diversity beta-group-significance   \
  --i-distance-matrix core-metrics-results/unweighted_unifrac_distance_matrix.qza   \
  --m-metadata-file metadata.tsv   --m-metadata-column Expermint   \
  --o-visualization core-metrics-results/unweighted-unifrac-expermint-title-significance.qzv   \
  --p-pairwise

Saved Visualization to: core-metrics-results/unweighted-unifrac-expermint-title-significance.qzv


#### Alpha group significance

In [18]:
%%bash
for metric in observed_features shannon pielou_e faith_pd
do
  qiime diversity alpha-group-significance \
    --i-alpha-diversity ${metric}_vector.qza \
    --m-metadata-file metadata.tsv \
    --o-visualization ${metric}_group_significance.qzv
done

Saved Visualization to: observed_features_group_significance.qzv
Saved Visualization to: shannon_group_significance.qzv
Saved Visualization to: pielou_e_group_significance.qzv
Saved Visualization to: faith_pd_group_significance.qzv


In [19]:
%%bash
for metric in observed_features shannon pielou_e
do
  qiime diversity alpha \
    --i-table rarefied_table.qza \
    --p-metric $metric \
    --o-alpha-diversity ${metric}_vector.qza
done

Saved SampleData[AlphaDiversity] to: observed_features_vector.qza
Saved SampleData[AlphaDiversity] to: shannon_vector.qza
Saved SampleData[AlphaDiversity] to: pielou_e_vector.qza


#### Alpha rarefaction plotting

In [21]:
!qiime diversity alpha-rarefaction \
  --i-table table.qza \
  --i-phylogeny rooted-tree.qza \
  --p-max-depth 41117 \
  --m-metadata-file metadata.tsv \
  --o-visualization alpha-rarefaction.qzv

Saved Visualization to: alpha-rarefaction.qzv


#### Beta Diversity

In [22]:
%%bash
for metric in jaccard braycurtis
do
  qiime diversity beta \
    --i-table rarefied_table.qza \
    --p-metric $metric \
    --o-distance-matrix ${metric}_distance_matrix.qza
done

Saved DistanceMatrix to: jaccard_distance_matrix.qza
Saved DistanceMatrix to: braycurtis_distance_matrix.qza


In [23]:
%%bash
for metric in unweighted_unifrac weighted_unifrac
do
  qiime diversity beta-phylogenetic \
    --i-table rarefied_table.qza \
    --i-phylogeny rooted-tree.qza \
    --p-metric $metric \
    --o-distance-matrix ${metric}_distance_matrix.qza
done

Saved DistanceMatrix to: unweighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: weighted_unifrac_distance_matrix.qza


#### PCoA

In [24]:
%%bash
for metric in jaccard braycurtis unweighted_unifrac weighted_unifrac
do
  qiime diversity pcoa \
    --i-distance-matrix ${metric}_distance_matrix.qza \
    --o-pcoa ${metric}_pcoa_results.qza
done

Saved PCoAResults to: jaccard_pcoa_results.qza
Saved PCoAResults to: braycurtis_pcoa_results.qza
Saved PCoAResults to: unweighted_unifrac_pcoa_results.qza
Saved PCoAResults to: weighted_unifrac_pcoa_results.qza


#### EMPeror

In [25]:
%%bash
for metric in jaccard braycurtis unweighted_unifrac weighted_unifrac
do
  qiime emperor plot \
    --i-pcoa ${metric}_pcoa_results.qza \
    --m-metadata-file metadata.tsv \
    --o-visualization ${metric}_emperor.qzv
done

Saved Visualization to: jaccard_emperor.qzv
Saved Visualization to: braycurtis_emperor.qzv
Saved Visualization to: unweighted_unifrac_emperor.qzv
Saved Visualization to: weighted_unifrac_emperor.qzv


#### Time series

In [28]:
!qiime emperor plot \
  --i-pcoa unweighted_unifrac_pcoa_results.qza \
  --m-metadata-file 10251/10251_prep_690_20201201-080233.txt \
  --p-custom-axes run_date \
  --o-visualization unweighted_unifrac_emperor_time.qzv

Plugin error from emperor:

  All values in a custom axis must be numeric, this summary shows the invalid values and the number of times they were found in column 'run_date':
  8/6/13     178
  5/15/13    113

Debug info has been saved to /var/folders/nb/_drlqg395x792y1_46fbz9lm0000gn/T/qiime2-q2cli-err-o_f13dio.log


Did not work in our stuy case



#### Beta rarefaction


In [27]:
!qiime diversity beta-rarefaction \
  --i-table table.qza \
  --i-phylogeny rooted-tree.qza\
  --p-metric unweighted_unifrac \
  --p-clustering-method nj \
  --p-sampling-depth 18003 \
  --m-metadata-file metadata.tsv \
  --o-visualization unweighted_unifrac_rarefaction.qzv

Saved Visualization to: unweighted_unifrac_rarefaction.qzv


#### PCoA biplot

In [28]:
!qiime feature-table relative-frequency \
  --i-table rarefied_table.qza \
  --o-relative-frequency-table relative_table.qza

Saved FeatureTable[RelativeFrequency] to: relative_table.qza


In [29]:
!qiime diversity pcoa-biplot \
  --i-pcoa unweighted_unifrac_pcoa_results.qza \
  --i-features relative_table.qza \
  --o-biplot unweighted_unifrac_pcoa_biplot.qza

Saved PCoAResults % Properties('biplot') to: unweighted_unifrac_pcoa_biplot.qza


In [30]:
!qiime emperor biplot \
  --i-biplot unweighted_unifrac_pcoa_biplot.qza \
  --m-sample-metadata-file metadata.tsv \
  --m-feature-metadata-file taxonomy.qza \
  --o-visualization unweighted_unifrac_emperor_biplot.qzv

Saved Visualization to: unweighted_unifrac_emperor_biplot.qzv


In [31]:
! qiime longitudinal volatility \
  --m-metadata-file ./metadata.tsv \
  --m-metadata-file ./core-metrics-results/unweighted_unifrac_pcoa_results.qza \
  --p-state-column Week \
  --p-individual-id-column sample_idi \
  --p-default-group-column 'Expermint' \
  --p-default-metric 'Axis 2' \
  --o-visualization ./pc_vol.qzv

Saved Visualization to: ./pc_vol.qzv


! qiime longitudinal first-distances \
  --i-distance-matrix ./core-metrics-results/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file ./metadata.tsv \
  --p-state-column Week \
  --p-individual-id-column sample id \
  --o-first-distances ./from_first_unifrac.qza

#### Mantel test¶


In [33]:
!qiime diversity mantel \
  --i-dm1 braycurtis_distance_matrix.qza \
  --i-dm2 weighted_unifrac_distance_matrix.qza \
  --o-visualization bc_vs_wu_mantel.qzv

Saved Visualization to: bc_vs_wu_mantel.qzv


#### Procrustes analysis¶


In [34]:
!qiime diversity procrustes-analysis \
  --i-reference braycurtis_pcoa_results.qza \
  --i-other weighted_unifrac_pcoa_results.qza \
  --o-transformed-reference braycurtis_pcoa_transformed.qza \
  --o-transformed-other weighted_unifrac_pcoa_transformed.qza

Saved PCoAResults to: braycurtis_pcoa_transformed.qza
Saved PCoAResults to: weighted_unifrac_pcoa_transformed.qza


In [35]:
!qiime emperor procrustes-plot \
  --i-reference-pcoa braycurtis_pcoa_transformed.qza \
  --i-other-pcoa weighted_unifrac_pcoa_transformed.qza \
  --m-metadata-file metadata.tsv \
  --o-visualization bc_vs_wu_procrustes.qzv

Saved Visualization to: bc_vs_wu_procrustes.qzv


#### Group significance

In [36]:
%%bash
for test in anosim permanova permdisp
do
  qiime diversity beta-group-significance \
    --i-distance-matrix unweighted_unifrac_distance_matrix.qza \
    --m-metadata-file metadata.tsv \
    --m-metadata-column "Expermint" \
    --p-method $test \
    --o-visualization unweighted_unifrac_body_site_${test}.qzv
done

Saved Visualization to: unweighted_unifrac_body_site_anosim.qzv
Saved Visualization to: unweighted_unifrac_body_site_permanova.qzv
Saved Visualization to: unweighted_unifrac_body_site_permdisp.qzv


#### Adonis test

In [37]:
!cat metadata.tsv | tr -d '-' > metadata_rev.txt


In [39]:
!qiime diversity adonis \
  --i-distance-matrix unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file metadata_rev.txt \
  --p-formula "Expermint" \
  --o-visualization unweighted_unifrac_body_site_adonis.qzv

Saved Visualization to: unweighted_unifrac_body_site_adonis.qzv


### Extracting data from QIIME 2 results

In [77]:
import pandas as pd

In [78]:
from qiime2 import Metadata


In [80]:
metaa = Visualozation.load('barplot.qzv')


NameError: name 'Visualozation' is not defined

In [43]:
meta

Metadata
--------
72 IDs x 10 columns
sample_idi:        ColumnProperties(type='categorical')
sample number:     ColumnProperties(type='numeric')
collection date:   ColumnProperties(type='categorical')
Week:              ColumnProperties(type='numeric')
Tempreture:        ColumnProperties(type='numeric')
Second Tempreture: ColumnProperties(type='numeric')
Expermint:         ColumnProperties(type='categorical')
Description:       ColumnProperties(type='categorical')
Material_type:     ColumnProperties(type='categorical')
temp:              ColumnProperties(type='categorical')

Call to_dataframe() for a tabular representation.

## visualization

In [1]:
from qiime2 import Visualization


In [45]:
df = meta.to_dataframe()
df.head()

,sample_idi,sample number,collection date,Week,Tempreture,Second Tempreture,Expermint,Description,Material_type,temp
sample id,,,,,,,,,,
ST6_1,ST6,1.0,12/11/2021,0.0,28.3,28.5,ST,Steel Test,Steel,warm
PT6_2,PT6,2.0,12/11/2021,0.0,30.2,30.3,PT,PVC Test,PVC,warm
CT6_3,CT6,3.0,12/11/2021,0.0,27.5,28.6,CT,Copper Test,Copper,warm
CC6_4,CC6,4.0,12/11/2021,0.0,22.3,22.1,CC,Copper Control,Copper,cold
PC6_5,PC6,5.0,12/11/2021,0.0,21.5,21.5,PC,PVC Control,PVC,cold


In [46]:
table_qzv = Visualization.load('table.qzv')

#### Faith Phylogenetic Diversity

In [2]:
faith_pd_group_significance = Visualization.load('core-metrics-results/faith-pd-group-significance.qzv')

In [3]:
faith_pd_group_significance

<visualization: Visualization uuid: 0967375e-f932-4ba2-b02d-5e89f7f013bc>

#### Rarefaction Visualization

In [20]:
rarefied_table = Visualization.load('rarefied_table.qzv')

#### Alpha and beta diversity analysis Visualization

In [4]:
unweighted_unifrac_emperor = Visualization.load('all_in_one/unweighted_unifrac_emperor.qzv')

In [33]:
unweighted_unifrac_emperor

<visualization: Visualization uuid: b6664f34-7e7e-49fc-a605-476064815a76>

In [5]:
weighted_unifrac_emperor = Visualization.load('all_in_one/weighted_unifrac_emperor.qzv')

In [34]:
weighted_unifrac_emperor

<visualization: Visualization uuid: c3dc3ffd-8992-4a3a-b6b6-b19517a07675>

In [6]:
jaccard_emperor = Visualization.load('all_in_one/jaccard_emperor.qzv')

In [7]:
bray_curtis_emperor = Visualization.load('all_in_one/bray_curtis_emperor.qzv')

In [35]:
bray_curtis_emperor

<visualization: Visualization uuid: fb5da5a4-6ff0-445c-a49a-e004dc0e3039>

#### Evenness metrics Visualization

In [36]:
evenness_group_significance = Visualization.load('core-metrics-results/evenness-group-significance.qzv')

In [37]:
evenness_group_significance

<visualization: Visualization uuid: b97ebdd6-90ea-4f5a-8f2f-40a96554a0f8>

#### Beta Diversity: unweighted UniFrac distances Visualization

In [9]:
unweighted_unifrac_expermint_title_significance = Visualization.load('core-metrics-results/unweighted-unifrac-expermint-title-significance.qzv')

In [38]:
unweighted_unifrac_expermint_title_significance

<visualization: Visualization uuid: 4d39d8c3-7407-44ec-85ad-f71660fa7638>

#### Alpha group significance Visualization
 

In [10]:
observed_features_group_significance = Visualization.load('observed_features_group_significance.qzv')

In [11]:
shannon_group_significance = Visualization.load('shannon_group_significance.qzv')

In [12]:
pielou_e_group_significance = Visualization.load('pielou_e_group_significance.qzv')

In [13]:
faith_pd_group_significance = Visualization.load('faith_pd_group_significance.qzv')

In [14]:
taxaa= Visualization.load('taxonomy.qzv')

In [15]:
baplott = Visualization.load('barplot.qzv')

In [39]:
baplott

<visualization: Visualization uuid: 3fb6df9c-466f-4d62-a3c9-fae367bd0061>

#### Alpha rarefaction plotting Visualization

In [18]:
alpha_rarefaction = Visualization.load('alpha-rarefaction.qzv')

In [19]:
alpha_rarefaction

<visualization: Visualization uuid: d61ac22e-5539-4b35-a4a4-801724dbb78b>

#### PCoA EMPeror Visualization
 

In [20]:
jaccard_emperor = Visualization.load('jaccard_emperor.qzv')

In [32]:
jaccard_emperor

<visualization: Visualization uuid: fa038226-d984-4688-8f56-31e7dc06c746>

In [21]:
braycurtis_emperor = Visualization.load('braycurtis_emperor.qzv')

In [22]:
unweighted_unifrac_emperor = Visualization.load('unweighted_unifrac_emperor.qzv')

In [23]:
weighted_unifrac_emperor = Visualization.load('weighted_unifrac_emperor.qzv')

#### Beta rarefaction Visualization

In [24]:
unweighted_unifrac_rarefaction = Visualization.load('unweighted_unifrac_rarefaction.qzv')

In [25]:
unweighted_unifrac_rarefaction

<visualization: Visualization uuid: 461a5111-f1ee-4eca-a06c-49bb818b54b1>

#### PCoA biplot


In [26]:
unweighted_unifrac_emperor_biplot = Visualization.load('unweighted_unifrac_emperor_biplot.qzv')

In [27]:
unweighted_unifrac_emperor_biplot

<visualization: Visualization uuid: ead91df5-6903-485a-bfee-03e95954ba9c>

#### Mantel test

In [28]:
bc_vs_wu_mantel = Visualization.load('bc_vs_wu_mantel.qzv')

In [31]:
bc_vs_wu_mantel

<visualization: Visualization uuid: 4eead68b-514e-4beb-a3df-a08478833e2a>

#### Procrustes analysis

In [29]:
bc_vs_wu_procrustes = Visualization.load('bc_vs_wu_procrustes.qzv')

In [30]:
bc_vs_wu_procrustes

<visualization: Visualization uuid: 55467d2e-0534-4f1e-a372-c2e91e3c08a0>